# Import packages / libraries

In [2]:
pip install py-solc-x

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# See reference for some code snippets for smart contracts and web3: https://coinsbench.com/how-to-deploy-and-interact-with-solidity-contracts-with-python-and-ganache-be63334323e6
# See also: https://medium.com/validitylabs/how-to-interact-with-the-ethereum-blockchain-and-create-a-database-with-python-and-sql-3dcbd579b3c0
# Import the Web3 object and establish an HTTP connection
from IPython.display import display, HTML , Image
import ipywidgets as widgets
from ipywidgets import Dropdown
# from PIL import Image
from web3 import Web3 
import requests
import getpass
import json
from solcx import compile_standard, install_solc
import termcolor
from IPython.display import Markdown, display

In [4]:
# ignore warnings if they occur 
import warnings
warnings.filterwarnings('ignore')

In [5]:
def printmd(string):
    display(Markdown(string))

In [6]:
# Connecting to the UZHETH Blockchain
web3 = Web3(Web3.HTTPProvider("http://localhost:8545"))

# Connection test
print(web3.isConnected())

# check if there are accounts 
print(web3.eth.accounts)

True
['0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26', '0x24174b610C359f4a4c099117e8aDB15E270C7cC0']


## Load Files 

In [7]:
# load ab and bytecode - both copied from remix 
# Interract with Smart contracts
# see also: https://medium.com/deepyr/interacting-with-ethereum-using-web3-py-and-jupyter-notebooks-e4207afa0085

# ABI code of the contract; Stored in a file (copied from Remix)
with open("Programming/22_HS/PancakeSwap/TokenSwap.abi") as tokenswap_abi_path: 
    swap_abi = tokenswap_abi_path.read()

# ABI code of the contract; Stored in a file (copied from Remix)
with open("Programming/22_HS/PancakeSwap/PancakeToken.abi") as token_abi_path: 
    token_abi = token_abi_path.read()

with open("Programming/22_HS/PancakeSwap/Swap.json") as s:
    swap_json = json.load(s)
    swap_bytecode = swap_json["object"]

with open("Programming/22_HS/PancakeSwap/Token.json") as t:
    token_json = json.load(t)
    token_bytecode = token_json["object"]
    
swap_contract = web3.eth.contract(abi=swap_abi, bytecode=swap_bytecode)
pancake_contract = web3.eth.contract(abi=token_abi, bytecode=token_bytecode)
uzh_contract = web3.eth.contract(abi=token_abi, bytecode=token_bytecode)

## Functions for later usage

In [8]:
# function to create tokens later 
def create_token(): 
    # unlock accounts 
    unlock(selected_user_1, password_1)
    unlock(selected_user_2, password_2)
    printmd("*Accounts unlocked*")
    # prepare pancake and uzh tokens on the accounts of both users as selected above
    tx_p = pancake_contract.constructor("Pancake Token", "Cake").transact({'from': selected_user_1})
    tx_u = uzh_contract.constructor("UZH Token", "UZHETH").transact({'from': selected_user_2})
    printmd("*Received receipt of Creating Cake Tokens*")
    tx_receipt_p = web3.eth.wait_for_transaction_receipt(tx_p)
    printmd("*Received receipt of Creating UZH-ETH Tokens*")
    tx_receipt_u = web3.eth.wait_for_transaction_receipt(tx_u)
    
    # save address of created tokens 
    cake_address = tx_receipt_p.contractAddress
    uzh_address = tx_receipt_u.contractAddress
    printmd("*Tokens created, address can be retrieved.*")
    return cake_address, uzh_address

# dropdown 
def handle_change():
    print(users_dropdown.value)
    
# add new user 
def new_user(number):
    description = 'Account ' + number
    
    dropdown = widgets.Dropdown(
    options=list_users,
    description=description,
    disabled=False)
    return dropdown

#toggle selection of Cake and UZH-ETH 
def selectToggle():
    select = widgets.ToggleButtons(
        options=['Cake', 'UZH-ETH'],
        description='Token to Swap:\t',
        disabled=False,
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    )
    return select

# unlock function to unlock account everytime we use it 
def unlock(account, password):
    web3.geth.personal.unlock_account(account, password)

# Start here 

## Step 1) Get Accounts

In [9]:
# get accounts 
list_users = web3.eth.accounts
printmd("### Here you can select the users to swap tokens.")
dropdown_1 = new_user("1")
display(dropdown_1)
dropdown_2 = new_user("2")
display(dropdown_2)

printmd("### Account 1, do you want to swap Cake tokens or UZH-ETH tokens?")
select_1 = selectToggle()
display(select_1)

### Here you can select the users to swap tokens.

Dropdown(description='Account 1', options=('0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26', '0x24174b610C359f4a4c…

Dropdown(description='Account 2', options=('0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26', '0x24174b610C359f4a4c…

### Account 1, do you want to swap Cake tokens or UZH-ETH tokens?

ToggleButtons(button_style='info', description='Token to Swap:\t', options=('Cake', 'UZH-ETH'), value='Cake')

In [10]:
# Get dropdown selection and selected tokens and map them 
# Goal: account 1 has cake, account 2 has uzh-eth 
select_1 = select_1.value

if select_1 == "Cake": 
    selected_user_1 = dropdown_1.value
    selected_user_2 = dropdown_2.value
    select_2 = "UZH-ETH"
    
elif select_1 == "UZH-ETH":
    select_1 = "Cake" 
    selected_user_1 = dropdown_2.value
    selected_user_2 = dropdown_1.value
    select_2 = "UZH-ETH"
    
print("Account", selected_user_1, "swaps tokens of", select_1)
print("Account", selected_user_2, "swaps tokens of", select_2)

Account 0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26 swaps tokens of Cake
Account 0x24174b610C359f4a4c099117e8aDB15E270C7cC0 swaps tokens of UZH-ETH


In [11]:
printmd("### Please input the passwords of the accounts.")

def get_password():
    password = getpass.getpass() 
    return password

    
passcheck_1 = False
passcheck_2 = False

printmd("Type in the password of the following account **" + str(selected_user_1) + "**:")
while passcheck_1 == False: 
    password_1 = get_password() 

    try: # check if passwords correct 
        unlock(selected_user_1, password_1)
        termcolor.cprint("Provided password is correct.",'green')
        passcheck_1 = True
        
    except:
        termcolor.cprint("Password wrong. try again.", "red")
        passcheck_1 = False  

printmd("---")
printmd("Type in the password of the following account **" + str(selected_user_2) + "**:")
while passcheck_2 == False:
    password_2 = get_password() 
    try: 
        unlock(selected_user_2, password_2)
        termcolor.cprint("Provided password is correct.",'green')
        passcheck_2 = True
        
    except:
        termcolor.cprint("Password wrong. try again.", "red")
        passcheck_2 = False  

### Please input the passwords of the accounts.

Type in the password of the following account **0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26**:

 ········


Provided password is correct.


---

Type in the password of the following account **0x24174b610C359f4a4c099117e8aDB15E270C7cC0**:

 ········


Provided password is correct.


In [12]:
printmd("### Start of Swapping")
printmd("**Are there existing token addresses of the tokens you want to swap?**")

existing = widgets.RadioButtons(
    options=['no', 'yes'],
    disabled=False)
display(existing)

### Start of Swapping

**Are there existing token addresses of the tokens you want to swap?**

RadioButtons(options=('no', 'yes'), value='no')

In [13]:
if existing.value == "yes": 
    printmd("Please input the token addresses of both Tokens, if you do not want to create new tokens.")
    cake_address = input("Please input address of Pancake Token:")
    uzh_address = input("Please input address of UZH-ETH Token:")

elif existing.value == "no": # make tokens, one on each a ccount 
    cake_address, uzh_address = create_token()

*Accounts unlocked*

*Received receipt of Creating Cake Tokens*

*Received receipt of Creating UZH-ETH Tokens*

*Tokens created, address can be retrieved.*

# Retrieve Tokens

In [14]:
# call token 
pancake_token = web3.eth.contract(cake_address, abi=token_abi)
uzh_token = web3.eth.contract(uzh_address, abi=token_abi)

printmd("-------\n")
printmd("**Cake Token Address:** &emsp;&emsp;&emsp;" + str(cake_address))
printmd("**UZH ETH Token Address:** &emsp;" + str(uzh_address))

printmd("-------\n")
printmd("### Account: " + str(selected_user_1))
printmd("**Current Amount Cake Token:** &emsp;&emsp;&emsp;" + str(pancake_token.functions.balanceOf(selected_user_1).call()//10**18))
printmd("**Current Amount UZH ETH Token:** &emsp;" + str(uzh_token.functions.balanceOf(selected_user_1).call()//10**18))

printmd("-------\n")
printmd("### Account: " + str(selected_user_2))
printmd("**Current Amount Cake Token:** &emsp;&emsp;&emsp;" + str(pancake_token.functions.balanceOf(selected_user_2).call()//10**18))
printmd("**Current Amount UZH ETH Token:** &emsp;" + str(uzh_token.functions.balanceOf(selected_user_2).call()//10**18))
printmd("-------")

-------


**Cake Token Address:** &emsp;&emsp;&emsp;0x0EaB0e3a7C15B0dE47C741147354Cc6E5d98dafc

**UZH ETH Token Address:** &emsp;0x8a0aE33AFeDb46D6210726da5573e27651Bf74c8

-------


### Account: 0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26

...

-------


### Account: 0x24174b610C359f4a4c099117e8aDB15E270C7cC0

**Current Amount Cake Token:** &emsp;&emsp;&emsp;0

**Current Amount UZH ETH Token:** &emsp;100

-------

# Step 2) Swap Smart Contract

In [15]:
# unlock accounts again to avoid errors 
unlock(selected_user_1, password_1)
unlock(selected_user_2, password_2)
printmd("*Accounts unlocked*")

# swap smart contract 
tx_s = swap_contract.constructor(cake_address, selected_user_1, uzh_address, selected_user_2).transact({
    'from': selected_user_2})
tx_receipt_s = web3.eth.wait_for_transaction_receipt(tx_s)
printmd("*Received receipt of initializing swap smart contract*")

*Accounts unlocked*

*Received receipt of initializing swap smart contract*

In [16]:
swap = web3.eth.contract(tx_receipt_s.contractAddress, abi=swap_abi)
swap_address = swap.address
printmd("**Swap Address:** " + str(swap_address))

**Swap Address:** 0x6d272EF3d6C2Cd3A98d6eeB7158b1DDf4088816A

In [17]:
tx_receipt_s

AttributeDict({'blockHash': HexBytes('0x32d6244030485ebc8b4072888125781c2a97b57833b5b5e52e695e8f6b264a0b'),
 'blockNumber': 3803173,
 'contractAddress': '0x6d272EF3d6C2Cd3A98d6eeB7158b1DDf4088816A',
 'cumulativeGasUsed': 725059,
 'effectiveGasPrice': 1000000000,
 'from': '0x24174b610C359f4a4c099117e8aDB15E270C7cC0',
 'gasUsed': 725059,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': None,
 'transactionHash': HexBytes('0x764cd67950baa85b078465badeb04c90bea585b318112625

# Step 3) Swap Tokens of Two Accounts

In [18]:
# calculate how much cake user 1 can swap with uzh from user 2 
max_cake = pancake_token.functions.balanceOf(selected_user_1).call()//10**18
max_uzh = uzh_token.functions.balanceOf(selected_user_2).call()//10**18

print("How much Cake Tokens do you want to swap?")
amount_pp = widgets.IntSlider(
    value=50,
    min=0,
    max=max_cake,
    step=1,
    description='Cake:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

display(amount_pp)

print("How much UZH ETH Tokens do you want to swap?")
amount_uu = widgets.IntSlider(
    value=50,
    min=0,
    max=max_uzh,
    step=1,
    description='UZH-ETH:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
display(amount_uu)

How much Cake Tokens do you want to swap?


IntSlider(value=50, continuous_update=False, description='Cake:', readout_format='.1f')

How much UZH ETH Tokens do you want to swap?


IntSlider(value=50, continuous_update=False, description='UZH-ETH:', readout_format='.1f')

In [19]:
amount_p = int(amount_pp.value)
amount_u = int(amount_uu.value)
printmd("You want to swap: <mark>**"+str(amount_p)+" Cake**</mark> Tokens with <mark>**" + str(amount_u) + " UZH ETH</mark>** Tokens")

You want to swap: <mark>**18 Cake**</mark> Tokens with <mark>**29 UZH ETH</mark>** Tokens

## Allowance

In [20]:
# pre-check allowance
possible_cake = pancake_token.functions.balanceOf(selected_user_1).call()//10**18
possible_uzh = uzh_token.functions.balanceOf(selected_user_2).call()//10**18
if amount_p > possible_cake: 
    printmd("<mark>PROBLEM</mark>") 
    print("You have not enough cake tokens to swap! Please try another number of tokens.") 
    
if amount_u > possible_uzh: 
    printmd("<mark>PROBLEM</mark>") 
    print("You have not enough UZH-ETH tokens to swap! Please try another number of tokens.") 

In [21]:
# unlock accounts with the given passwords 
printmd("### Allowance")

unlock(selected_user_1, password_1)
unlock(selected_user_2, password_2)
printmd("*Accounts unlocked.*")

# approve to swap given number of tokens   
printmd("**Approve Pancake Token Swap**") 
printmd("*Approved.*") 
approve_p = pancake_token.functions.approve(swap_address, amount_p * 10**18).transact({'from': selected_user_1})
approve_p_receipt = web3.eth.wait_for_transaction_receipt(approve_p)
print("Current Amount Cake Token allowed to swap:\t",pancake_token.functions.allowance(selected_user_1, swap_address).call()//10**18)

print("\n")
printmd("**Approve UZH ETH Token Swap**")
printmd("*Approved.*") 
approve_u = uzh_token.functions.approve(swap_address, amount_u * 10**18).transact({'from': selected_user_2})
approve_u_receipt = web3.eth.wait_for_transaction_receipt(approve_u)
print("Current Amount UZH ETH Token allowed to swap:\t",uzh_token.functions.allowance(selected_user_2, swap_address).call()//10**18)

### Allowance

*Accounts unlocked.*

**Approve Pancake Token Swap**

*Approved.*

Current Amount Cake Token allowed to swap:	 18




**Approve UZH ETH Token Swap**

*Approved.*

Current Amount UZH ETH Token allowed to swap:	 29


In [24]:
# unlock accounts with the given passwords 
unlock(selected_user_1, password_1)
unlock(selected_user_2, password_2)
printmd("*Accounts unlocked.*")
tx_swap_tok = swap.functions.swap(amount_p * 10**18, amount_u * 10**18).transact({'from': selected_user_1, 'to': swap_address})
tx_receipt_swap_tok = web3.eth.wait_for_transaction_receipt(tx_swap_tok)
printmd("*Received receipt of swap transaction*")
printmd("Transaction done, swap successfully executed")

*Accounts unlocked.*

*Received receipt of swap transaction*

Transaction done, swap successfully executed

In [30]:
printmd("-------\n")
printmd("### Account: " + str(selected_user_1))
printmd("**Current Amount Cake Token:** &emsp;&emsp;&emsp;" + str(pancake_token.functions.balanceOf(selected_user_1).call()//10**18))
printmd("**Current Amount UZH ETH Token:** &emsp;" + str(uzh_token.functions.balanceOf(selected_user_1).call()//10**18))

printmd("-------\n")
printmd("### Account: " + str(selected_user_2))
printmd("**Current Amount Cake Token:** &emsp;&emsp;&emsp;" + str(pancake_token.functions.balanceOf(selected_user_2).call()//10**18))
printmd("**Current Amount UZH ETH Token:** &emsp;" + str(uzh_token.functions.balanceOf(selected_user_2).call()//10**18))
printmd("-------")

printmd("If you want to swap again, you can start again at **step 3** if you have enough tokens. \
    The pre-check allowance part will give you an error message if you have not enough tokens left. \
    You can also start from **step 1** and input the token addresses by yourself without \
    creating new tokens. This way, you can **change the swap direction**, \
    if you have some amount of both tokens.")
print("Cake Address:", cake_address)
print("UZH-ETH Address:", uzh_address)

-------


### Account: 0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26

**Current Amount Cake Token:** &emsp;&emsp;&emsp;82

**Current Amount UZH ETH Token:** &emsp;29

-------


...

**Current Amount Cake Token:** &emsp;&emsp;&emsp;18

**Current Amount UZH ETH Token:** &emsp;71

-------

If you want to swap again, you can start again at **step 3** if you have enough tokens.     The pre-check allowance part will give you an error message if you have not enough tokens left.     You can also start from **step 1** and input the token addresses by yourself without     creating new tokens. This way, you can **change the swap direction**,     if you have some amount of both tokens.

Cake Address: 0x0EaB0e3a7C15B0dE47C741147354Cc6E5d98dafc
UZH-ETH Address: 0x8a0aE33AFeDb46D6210726da5573e27651Bf74c8
